# A simple introduction to 3d rendering

You'll need 2 things: pyetk (conda install -c conda-forge pyevtk) and paraview (https://www.paraview.org/download/)

# 1. Points data

In [1]:
from pyevtk.hl import pointsToVTK
import numpy as np


npoints = 10

for t in range(0,10):
    x = np.random.rand(npoints)
    y = np.random.rand(npoints)
    z = np.random.rand(npoints)

    mass = np.ones(npoints)

    pointsToVTK("./points_"+str(t), x, y, z, data = {"mass" : mass})

# 2. Volume data

In [2]:
from pyevtk.hl import gridToVTK
import random as rnd

# Dimensions
nx, ny, nz = 64, 64, 64
lx, ly, lz = 32.0, 32.0, 32.0
dx, dy, dz = lx/nx, ly/ny, lz/nz


# Coordinates
X = np.linspace(-lx/2,lx/2, nx)
Y = np.linspace(-ly/2,ly/2, nx)
Z = np.linspace(-lz/2,lz/2, nx)
x = np.zeros((nx, ny, nz))
y = np.zeros((nx, ny, nz))
z = np.zeros((nx, ny, nz))

for k in range(nz):
    for j in range(ny):
        for i in range(nx): 
            x[i,j,k] = X[i] 
            y[i,j,k] = Y[j] 
            z[i,j,k] = Z[k]
                

def harmonic_trap_potential(x,y,z,theta=45):
    kx=1
    ky=0.2
    kz=1
    return - 1/2 *((kx*(x*np.cos(theta)-y*np.sin(theta))**2)+
                     (ky*(x*np.sin(theta)+y*np.cos(theta))**2) + 1/2*kz*z**2)


potential = np.zeros((nx,ny,nz))

for k in range(nz):
    for j in range(ny):
        for i in range(nx):
            potential[i,j,k] = harmonic_trap_potential(x[i,j,k],y[i,j,k],z[i,j,k])
            
# Variables
gridToVTK("./potential", x, y, z, pointData = {"potential" : potential})

'C:\\Users\\nunoa\\PEEC 2021\\PEEC_2021_OT\\potential.vts'

# 3. Vector Field data

In [3]:
from pyevtk.hl import gridToVTK
import random as rnd
# Dimensions
nx, ny, nz = 64, 64, 64
lx, ly, lz = 32.0, 32.0, 32.0
dx, dy, dz = lx/nx, ly/ny, lz/nz


# Coordinates
X = np.linspace(-lx/2,lx/2, nx)
Y = np.linspace(-ly/2,ly/2, nx)
Z = np.linspace(-lz/2,lz/2, nx)
x = np.zeros((nx, ny, nz))
y = np.zeros((nx, ny, nz))
z = np.zeros((nx, ny, nz))

for k in range(nz):
    for j in range(ny):
        for i in range(nx): 
            x[i,j,k] = X[i] 
            y[i,j,k] = Y[j] 
            z[i,j,k] = Z[k]
                

def harmonic_trap(x,y,z,theta=45):
    kx=1
    ky=0.2
    kz=1
    
    Fx = - (kx*(x*np.cos(theta)-y*np.sin(theta))*np.cos(theta)+(ky*(x*np.sin(theta)+y*np.cos(theta))*np.sin(theta)))
    Fy = - (-kx*(x*np.cos(theta)-y*np.sin(theta))*np.sin(theta)+(ky*(x*np.sin(theta)+y*np.cos(theta))*np.cos(theta)))
    Fz = - kz*z
    return Fx,Fy,Fz


Fx = np.zeros((nx,ny,nz))
Fy = np.zeros((nx,ny,nz))
Fz = np.zeros((nx,ny,nz))
for k in range(nz):
    for j in range(ny):
        for i in range(nx):
            fx,fy,fz = harmonic_trap(x[i,j,k],y[i,j,k],z[i,j,k])
            Fx[i,j,k] = fx
            Fy[i,j,k] = fy
            Fz[i,j,k] = fz
            
# Variables
gridToVTK("./Force", x, y, z, pointData = {"force" : (Fx,Fy,Fz)})

'C:\\Users\\nunoa\\PEEC 2021\\PEEC_2021_OT\\Force.vts'

In [4]:
import numpy as np
from matplotlib.pyplot import *
%matplotlib notebook
from scipy import *
from tqdm.notebook import *
from matplotlib.animation import *

In [5]:
#########################
#numerical integrator####


def integrate_euler(F,t,y,tstop,dt):

    def euler(F,t,y,dt):
        #retorna o incremento apra utilizar na resolucao
        K0=dt*F(t,y)
        return K0

    T=[]
    Y=[]
    T.append(t)
    Y.append(y)
    
    pbar = tqdm(total=(tstop-dt))
    while t<tstop:
        pbar.update(dt)
        dt=min(dt,tstop-t)
        y=y+euler(F,t,y,dt)
        t=t+dt
        T.append(t)
        Y.append(y)

    return np.array(T),np.array(Y) 


##################################
#functions for simulating the dynamics

def harmonic_trap_potential(x,y,z,k,theta):
    kx=k[0]
    ky=k[1]
    kz=k[2]
    return - 1/2 *((kx*(x*np.cos(theta)-y*np.sin(theta))**2)+
                     (ky*(x*np.sin(theta)+y*np.cos(theta))**2) + 1/2*kz*z**2)

def harmonic_trap(x,y,z,k,theta):

    
    kx=k[0]
    ky=k[1]
    kz=k[2]
    
    Fx = - (kx*(x*np.cos(theta)-y*np.sin(theta))*np.cos(theta)+(ky*(x*np.sin(theta)+y*np.cos(theta))*np.sin(theta)))
    Fy = - (-kx*(x*np.cos(theta)-y*np.sin(theta))*np.sin(theta)+(ky*(x*np.sin(theta)+y*np.cos(theta))*np.cos(theta)))
    Fz = - kz*z
    return Fx,Fy,Fz

def brownian_simulation(t_stop, dt, k, W, gamma, theta = 0, m=1,delta_x=0,delta_v=0):
    print('k',k)
    print('theta',theta)
    print('W',W)
    def F(t,y):

        Fi=np.zeros(6) #[x,vx,y,vy,z,vz]
        Fx,Fy,Fz = harmonic_trap(y[0],y[2],y[4],k,theta)
        
        #normalization factor of the noise - Variance of the noise must be equal to 1/dt
        W1 = W/np.sqrt(dt)
        
        Fi[0]=y[1] 
        Fi[1]= -(gamma/m) * y[1] + Fx/m   + W1/m*(np.random.normal(scale=1))
        
        Fi[2]=y[3]  
        Fi[3]= -(gamma/m) * y[3] + Fy/m  + W1/m*(np.random.normal(scale=1))
        
        Fi[4]=y[5] 
        Fi[5]= -(gamma/m) * y[5] + Fz/m + W1/m*(np.random.normal(scale=1))
            
        return Fi

    
    def initial_condition():
        y=np.zeros(6)

        for i in range(0,6,2):

            y[i] = delta_x*(1-2*np.random.rand())
            y[i+1] = delta_v*(1-2*np.random.rand())

        #define initial position
        y[0]= 10
        y[2]= 10
        y[4]= 10
        
        return y
    
    r0=initial_condition()
    T,Y=integrate_euler(F,0,r0,t_stop,dt)
    return T,Y


In [6]:
#[kx,ky,kz] - parameters of the harmonic trap
k_0 = [.9,.2,.5]

#noise parameters
#drag parameter
m=1
gamma_0 = .2
kB = 1
T0 = 1
D = kB * T0 / gamma_0
W_0 = np.sqrt(2*kB*T0*gamma_0)


#xy angle
theta = 50


#simulation time, dt step
t_stop = 1000
dt = 0.01

#simulation Y is a vector with [x,vx,y,vy,z,vz](t)
initial_pos = []
T,Y = brownian_simulation(t_stop,dt, k_0, W_0, gamma = gamma_0, m = m , theta = theta, delta_x=2)

time = T
x_data = Y[:,0]
y_data = Y[:,2] 
z_data = Y[:,4]

k [0.9, 0.2, 0.5]
theta 50
W 0.6324555320336759


  0%|          | 0/999.99 [00:00<?, ?it/s]

In [8]:
# save data from the simulation to paraview files

#particle position data#######################

for t in range(0,len(time),10):
    x = x_data[t]
    y = y_data[t]
    z = z_data[t]

    mass = np.ones(1)

    pointsToVTK("./paraview_data1/particle_t_"+str(t), x, y, z, data = {"mass" : mass})
    
#potential data###############################

# Dimensions
nx, ny, nz = 64, 64, 64
lx, ly, lz = np.abs(x_data).max(), np.abs(y_data).max(), np.abs(z_data).max()
dx, dy, dz = lx/nx, ly/ny, lz/nz
# Coordinates
X = np.linspace(-lx,lx, nx)
Y = np.linspace(-ly,ly, ny)
Z = np.linspace(-lz,lz, nz)
x = np.zeros((nx, ny, nz))
y = np.zeros((nx, ny, nz))
z = np.zeros((nx, ny, nz))

for k in range(nz):
    for j in range(ny):
        for i in range(nx): 
            x[i,j,k] = X[i] 
            y[i,j,k] = Y[j] 
            z[i,j,k] = Z[k]
            

potential = np.zeros((nx,ny,nz))
for k in range(nz):
    for j in range(ny):
        for i in range(nx):
            potential[i,j,k] = harmonic_trap_potential(x[i,j,k],y[i,j,k],z[i,j,k],k_0, theta)
            
gridToVTK("./paraview_data1/potential_trap", x, y, z, pointData = {"potential" : potential})

#force field data#############################

Fx = np.zeros((nx,ny,nz))
Fy = np.zeros((nx,ny,nz))
Fz = np.zeros((nx,ny,nz))
for k in range(nz):
    for j in range(ny):
        for i in range(nx):
            fx,fy,fz = harmonic_trap(x[i,j,k],y[i,j,k],z[i,j,k],k_0,theta)
            Fx[i,j,k] = fx
            Fy[i,j,k] = fy
            Fz[i,j,k] = fz
            
# Variables
gridToVTK("./paraview_data1/Force_field", x, y, z, pointData = {"force" : (Fx,Fy,Fz)})




'C:\\Users\\nunoa\\PEEC 2021\\PEEC_2021_OT\\paraview_data1\\Force_field.vts'